In [ ]:
import pandas as pd
import numpy as np
import yaml

In [ ]:
# Ruta a tu archivo YAML
ruta = r"C:\Users\scerda\Documents\langgraph-postgres-energy-data\src\agent\schema_context.yaml"

# Leer el archivo como string y convertirlo a dict
with open(ruta, 'r', encoding='utf-8') as f:
    schema = yaml.safe_load(f)

In [2]:
# get current date
from datetime import datetime
from datetime import timedelta

date = datetime.now()

In [ ]:
prompt = f"""You are a SQL expert and Data Analyst tasked with generating user questions and sql queries regarding a energy consumption `postgresql` database.
Follow these strict guidelines:

1. Write syntactically correct SQL for the `postgresql` dialect.  
2. Use **only** the tables and columns provided below.  
3. Do **not** use `SELECT *`; always specify only the relevant columns.  
4. Include `JOIN`s when necessary, using primary keys appropriately.  
5. Use proper identifier quoting based on the `postgresql` syntax.  
6. Always prefix every table with its schema name.  
7. Do **not** make assumptions or use any data that is not present in the table definitions.  
8. If the question mentions an entity or name, assume it refers to a building name.  
9. Preferred sources, depending on the requested granularity and time span:  
   • Monthly data ­→ use **smart_buildings.energy_consumption_monthly_metrics**.  
   • Weekly data ­→ use **smart_buildings.energy_consumption_weekly_metrics**. 
10. If the user makes a query for a building without specifying a period, generate a query with all the metrics of the current month.

CURRENT DATE: {date}

{schema}

Use the following table as reference:

Return **only** the a JSON with SQL query or List of queries no additional explanation or formatting. The JSon needs to be in a format to later on by transform into a dataframe

"""

In [5]:
from openai import OpenAI

client = OpenAI()

response = client.responses.create(
  model="gpt-4.1",
  input=prompt
)
print(response)




Response(id='resp_681a3959281c8191bae7d91fa161024f0c1d236cddfba1a9', created_at=1746549081.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-2025-04-14', object='response', output=[ResponseOutputMessage(id='msg_681a3959c5948191886838ab4bb8551f0c1d236cddfba1a9', content=[ResponseOutputText(annotations=[], text='{\n  "queries": [\n    {\n      "sql": "SELECT b.\\"name\\" AS building_name, m.\\"year_month\\", m.\\"total_consumption_kwh\\", m.\\"total_consumption_prev_month_kwh\\", m.\\"diff_pct_consumption_prev_month\\" FROM smart_buildings.\\"building\\" b JOIN smart_buildings.\\"energy_consumption_monthly_metrics\\" m ON b.\\"cups\\" = m.\\"cups\\" WHERE b.\\"name\\" = \'X\' AND m.\\"year_month\\" = DATE_TRUNC(\'month\', CURRENT_DATE) ORDER BY m.\\"year_month\\" DESC LIMIT 1;"\n    },\n    {\n      "sql": "SELECT b.\\"name\\" AS building_name, m.\\"year_month\\", m.\\"total_consumption_kwh\\", m.\\"total_consumption_prev_year_same_month_kwh\\", (m.\\

In [18]:
print(response.output_text)

{
  "queries": [
    {
      "sql": "SELECT b.\"name\" AS building_name, m.\"year_month\", m.\"total_consumption_kwh\", m.\"total_consumption_prev_month_kwh\", m.\"diff_pct_consumption_prev_month\" FROM smart_buildings.\"building\" b JOIN smart_buildings.\"energy_consumption_monthly_metrics\" m ON b.\"cups\" = m.\"cups\" WHERE b.\"name\" = 'X' AND m.\"year_month\" = DATE_TRUNC('month', CURRENT_DATE) ORDER BY m.\"year_month\" DESC LIMIT 1;"
    },
    {
      "sql": "SELECT b.\"name\" AS building_name, m.\"year_month\", m.\"total_consumption_kwh\", m.\"total_consumption_prev_year_same_month_kwh\", (m.\"total_consumption_kwh\" - m.\"total_consumption_prev_year_same_month_kwh\") AS diff_kwh FROM smart_buildings.\"building\" b JOIN smart_buildings.\"energy_consumption_monthly_metrics\" m ON b.\"cups\" = m.\"cups\" WHERE b.\"name\" = 'X' AND m.\"year_month\" = DATE_TRUNC('month', CURRENT_DATE) ORDER BY m.\"year_month\" DESC LIMIT 1;"
    },
    {
      "sql": "SELECT b.\"name\" AS building_

| Pregunta                                                                                       | Consulta SQL                                                                                                                                                                                                                                                                                                                                                      |
|------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| ¿Qué diferencia hubo en el consumo del edificio X entre este mes y el mes anterior?            | `SELECT b."name" AS building_name, m."year_month", m."total_consumption_kwh", m."total_consumption_prev_month_kwh", m."diff_pct_consumption_prev_month" FROM smart_buildings."building" b JOIN smart_buildings."energy_consumption_monthly_metrics" m ON b."cups" = m."cups" WHERE b."name" = 'X' AND m."year_month" = DATE_TRUNC('month', CURRENT_DATE) ORDER BY m."year_month" DESC LIMIT 1;` |
| ¿Cómo cambió el consumo energético del edificio X este mes respecto al mismo mes del año pasado? | `SELECT b."name" AS building_name, m."year_month", m."total_consumption_kwh", m."total_consumption_prev_year_same_month_kwh", (m."total_consumption_kwh" - m."total_consumption_prev_year_same_month_kwh") AS diff_kwh FROM smart_buildings."building" b JOIN smart_buildings."energy_consumption_monthly_metrics" m ON b."cups" = m."cups" WHERE b."name" = 'X' AND m."year_month" = DATE_TRUNC('month', CURRENT_DATE) ORDER BY m."year_month" DESC LIMIT 1;` |
| ¿Cuál ha sido el consumo total del edificio X en marzo 2024?                                   | `SELECT b."name" AS building_name, m."year_month", m."total_consumption_kwh" FROM smart_buildings."building" b JOIN smart_buildings."energy_consumption_monthly_metrics" m ON b."cups" = m."cups" WHERE b."name" = 'X' AND m."year_month" = DATE '2024-03-01';`                                                                                                 |
| ¿Y cómo se compara con el mes anterior? (requiere que el sistema entienda contexto previo)     | `SELECT b."name" AS building_name, m."year_month", m."total_consumption_prev_month_kwh" FROM smart_buildings."building" b JOIN smart_buildings."energy_consumption_monthly_metrics" m ON b."cups" = m."cups" WHERE b."name" = 'X' AND m."year_month" = DATE '2024-03-01';`                                                                                      |
| ¿Cuál es el consumo acumulado (YTD) del edificio X en lo que va del año?                       | `SELECT b."name" AS building_name, m."year_month", m."ytd_consumption_kwh" FROM smart_buildings."building" b JOIN smart_buildings."energy_consumption_monthly_metrics" m ON b."cups" = m."cups" WHERE b."name" = 'X' AND m."year_month" = DATE_TRUNC('month', CURRENT_DATE) ORDER BY m."year_month" DESC LIMIT 1;`                                                |
| ¿Y cómo se compara ese YTD con el año pasado en la misma fecha?                                | `SELECT b."name" AS building_name, m."year_month", m."ytd_consumption_kwh", m."ytd_prev_year_consumption_kwh" FROM smart_buildings."building" b JOIN smart_buildings."energy_consumption_monthly_metrics" m ON b."cups" = m."cups" WHERE b."name" = 'X' AND m."year_month" = DATE_TRUNC('month', CURRENT_DATE) ORDER BY m."year_month" DESC LIMIT 1;`             |


In [ ]:
[
  {
    "pregunta": "¿Qué diferencia hubo en el consumo del edificio Hospital Margarita entre este mes y el mes anterior?",
    "relevant_values":{"name":["Hospital Santa Margarita","Hospital Maria"]},
    "consulta_sql": "SELECT b.\"name\" AS building_name, m.\"year_month\", m.\"total_consumption_kwh\", m.\"total_consumption_prev_month_kwh\", m.\"diff_pct_consumption_prev_month\" FROM smart_buildings.\"building\" b JOIN smart_buildings.\"energy_consumption_monthly_metrics\" m ON b.\"cups\" = m.\"cups\" WHERE b.\"name\" = 'Hospital Santa Margarita' AND m.\"year_month\" = DATE_TRUNC('month', CURRENT_DATE) ORDER BY m.\"year_month\" DESC LIMIT 1;"
  },
  {
    "pregunta": "¿Cómo cambió el consumo energético del edificio Escola de Arboles este mes respecto al mismo mes del año pasado?",
    "relevant_values":{"name":["Escuela de Arboles","Escuala Augusto"]},
    "consulta_sql": "SELECT b.\"name\" AS building_name, m.\"year_month\", m.\"total_consumption_kwh\", m.\"total_consumption_prev_year_same_month_kwh\", (m.\"total_consumption_kwh\" - m.\"total_consumption_prev_year_same_month_kwh\") AS diff_kwh FROM smart_buildings.\"building\" b JOIN smart_buildings.\"energy_consumption_monthly_metrics\" m ON b.\"cups\" = m.\"cups\" WHERE b.\"name\" = 'Escuela de Arboles' AND m.\"year_month\" = DATE_TRUNC('month', CURRENT_DATE) ORDER BY m.\"year_month\" DESC LIMIT 1;"
  },
  {
    "pregunta": "¿Cuál ha sido el consumo total del edificio Escuela de Girona en marzo 2024?",
     "relevant_values":{"name":["Escola de Girona","Escuela de Ginebra"]},
    "consulta_sql": "SELECT b.\"name\" AS building_name, m.\"year_month\", m.\"total_consumption_kwh\" FROM smart_buildings.\"building\" b JOIN smart_buildings.\"energy_consumption_monthly_metrics\" m ON b.\"cups\" = m.\"cups\" WHERE b.\"name\" = 'Escola de Girona' AND m.\"year_month\" = DATE '2024-03-01';"
  },
  {
    "pregunta": "¿Y cómo se compara con el mes anterior?",
    "relevant_values":{"name":[]},
    "consulta_sql": "SELECT b.\"name\" AS building_name, m.\"year_month\", m.\"total_consumption_prev_month_kwh\" FROM smart_buildings.\"building\" b JOIN smart_buildings.\"energy_consumption_monthly_metrics\" m ON b.\"cups\" = m.\"cups\" WHERE b.\"name\" = 'Escola de Girona' AND m.\"year_month\" = DATE '2024-03-01';"
  },
  {
    "pregunta": "¿Cuál es el consumo acumulado (YTD) del edificio Imperial Norte en lo que va del año?",
    "relevant_values":{"name":["Imperial Norte"]},
    "consulta_sql": "SELECT b.\"name\" AS building_name, m.\"year_month\", m.\"ytd_consumption_kwh\" FROM smart_buildings.\"building\" b JOIN smart_buildings.\"energy_consumption_monthly_metrics\" m ON b.\"cups\" = m.\"cups\" WHERE b.\"name\" = 'Imperial Norte' AND m.\"year_month\" = DATE_TRUNC('month', CURRENT_DATE) ORDER BY m.\"year_month\" DESC LIMIT 1;"
  },
  {
    "pregunta": "¿Y cómo se compara ese YTD con el año pasado en la misma fecha?",
    "consulta_sql": "SELECT b.\"name\" AS building_name, m.\"year_month\", m.\"ytd_consumption_kwh\", m.\"ytd_prev_year_consumption_kwh\" FROM smart_buildings.\"building\" b JOIN smart_buildings.\"energy_consumption_monthly_metrics\" m ON b.\"cups\" = m.\"cups\" WHERE b.\"name\" = 'Imperial Norte' AND m.\"year_month\" = DATE_TRUNC('month', CURRENT_DATE) ORDER BY m.\"year_month\" DESC LIMIT 1;"
  },
    {
  "pregunta":"Cual es el edificio con mayor consumo en abril y cuanto cambio en comparación con el mes anterior?",
  "relevant_values":{"name":[]},
  "consulta_sql":"SELECT b.\"name\" AS building_name, m.\"year_month\", m.\"total_consumption_kwh\", m.\"total_consumption_prev_month_kwh\", (m.\"total_consumption_kwh\" - m.\"total_consumption_prev_month_kwh\") AS diff_kwh FROM smart_buildings.\"building\" b JOIN smart_buildings.\"energy_consumption_monthly_metrics\" m ON b.\"cups\" = m.\"cups\" WHERE m.\"year_month\" = DATE '2024-04-01' ORDER BY m.\"total_consumption_kwh\" DESC LIMIT 1;"
  },
    {
    "pregunta":"Cual es el edificio con menor consumo en abril y cuanto cambio en comparación con el mes anterior?",
    "relevant_values":{"name":[]},
    "consulta_sql":"SELECT b.\"name\" AS building_name, m.\"year_month\", m.\"total_consumption_kwh\", m.\"total_consumption_prev_month_kwh\", (m.\"total_consumption_kwh\" - m.\"total_consumption_prev_month_kwh\") AS diff_kwh FROM smart_buildings.\"building\" b JOIN smart_buildings.\"energy_consumption_monthly_metrics\" m ON b.\"cups\" = m.\"cups\" WHERE m.\"year_month\" = DATE '2024-04-01' ORDER BY m.\"total_consumption_kwh\" ASC LIMIT 1;"
  },
]
